In [1]:
import pickle
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome import options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from random import randint
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from  selenium.webdriver.common.by import By
import csv
import json
import pymongo
from pymongo import MongoClient
import re
import win32clipboard


from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
s = Service("C:/Users/armi1/PycharmProjects/pythonProject/venv/crawl_data_facebook\chromedriver.exe")
browser = webdriver.Chrome(service=s)
# mo thu 1 trang web
browser.get("https://www.linkedin.com/checkpoint/rm/sign-in-another-account?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin")


In [3]:
def login_linkdin():
    credential = open('login.txt')
    line = credential.readlines()
    username = line[0]
    password = line[1]
    # import user name
    email_field = browser.find_element(By.ID,'username')
    email_field.send_keys(username)
    sleep(randint(1,3))
    # import password
    password_login = browser.find_element(By.ID,'password')
    password_login.send_keys(password)
    sleep(randint(1,3))
    #key in ueser name
    login_field = browser.find_element(By.XPATH,"/html/body/div/main/div[2]/div[1]/form/div[3]/button")
    login_field.click()
login_linkdin()
sleep(2)

In [6]:
key_word = 'natural language processing'
def search_linkdin():
    x= browser.find_element(By.CLASS_NAME,'global-nav__content')
    x.click()
    sleep(2)
    click_findkey = browser.find_element(By.XPATH,'//*[@id="global-nav-search"]/div/button')
    click_findkey.click()
    sleep(3)
    search_line = browser.find_element(By.XPATH,'//*[@id="global-nav-typeahead"]/input')
    sleep(2)
    search_line.send_keys(key_word)
    sleep(3)
    search_line.send_keys(Keys.RETURN)
    sleep(4)
    Post = browser.find_element(By.XPATH,'//*[@aria-label="Posts"]').click()
    sleep(2)
#     browser.find_element(By.XPATH,'/html/body/div[5]/aside/section/header/div[3]/button[2]/li-icon').click()

search_linkdin()


In [502]:

def get_url_profile():
    all_url_post = []
    #CLick vao cac post de lay the class       
    sleep(1)
    recentList = browser.find_elements(By.CLASS_NAME,"reusable-search__result-container")
    url_web = browser.current_url
    sleep(1)

    for list in recentList :
        list.click()
        sleep(2)
        url_web_click = browser.current_url
        
        if url_web_click != url_web:
            sleep(2)
            browser.back()
            
            #click lay link url post
            browser.find_element(By.CLASS_NAME,'feed-shared-control-menu__trigger').click()
            sleep(1)
            browser.find_element(By.CLASS_NAME,'feed-shared-control-menu__item.option-share-via').click()
            sleep(2)
            #tat clipboard de pass qua err
            browser.find_element(By.CLASS_NAME,'artdeco-button__icon').click()
            sleep(1)
            #lay du lieu ra tu clipboard
            win32clipboard.OpenClipboard()
            data = win32clipboard.GetClipboardData()
            win32clipboard.CloseClipboard()
            all_url_post.append(data)
        else:
            #click lay link url post
            browser.find_element(By.CLASS_NAME,'feed-shared-control-menu__trigger').click()
            sleep(1)
            browser.find_element(By.CLASS_NAME,'feed-shared-control-menu__item.option-share-via').click()
            sleep(2)
            #tat clipboard de pass qua err
            browser.find_element(By.CLASS_NAME,'artdeco-button__icon').click()
            sleep(1)
            #lay du lieu ra tu clipboard
            win32clipboard.OpenClipboard()
            data = win32clipboard.GetClipboardData()
            win32clipboard.CloseClipboard()
            all_url_post.append(data)
    return all_url_post
# get_url_profile()

In [514]:
number_page = 10
def get_url_all_page():
    url_all_page = []
    for i in range(number_page):
        url_one_page =  get_url_profile()
        sleep(2)
        try:
            element = browser.find_element(By.CLASS_NAME,'search-marvel-srp__content-detail-bottom-divider')
            element.location_once_scrolled_into_view
            sleep(1)
            browser.find_element(By.XPATH,f'//*[@aria-label="Page {i+2}"]').click()
            sleep(1.5)
        except:
            pass
        for url in url_one_page:
                url_all_page.append(url)
        sleep(1)

    return url_all_page
url_all_page = get_url_all_page()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".feed-shared-control-menu__trigger"}
  (Session info: chrome=96.0.4664.45)
Stacktrace:
Backtrace:
	Ordinal0 [0x008D6903+2517251]
	Ordinal0 [0x0086F8E1+2095329]
	Ordinal0 [0x00772848+1058888]
	Ordinal0 [0x0079D448+1233992]
	Ordinal0 [0x0079D63B+1234491]
	Ordinal0 [0x007C7812+1406994]
	Ordinal0 [0x007B650A+1336586]
	Ordinal0 [0x007C5BBF+1399743]
	Ordinal0 [0x007B639B+1336219]
	Ordinal0 [0x007927A7+1189799]
	Ordinal0 [0x00793609+1193481]
	GetHandleVerifier [0x00A65904+1577972]
	GetHandleVerifier [0x00B10B97+2279047]
	GetHandleVerifier [0x00966D09+534521]
	GetHandleVerifier [0x00965DB9+530601]
	Ordinal0 [0x00874FF9+2117625]
	Ordinal0 [0x008798A8+2136232]
	Ordinal0 [0x008799E2+2136546]
	Ordinal0 [0x00883541+2176321]
	BaseThreadInitThunk [0x76E36739+25]
	RtlGetFullPathName_UEx [0x77D18AFF+1215]
	RtlGetFullPathName_UEx [0x77D18ACD+1165]


In [425]:
def crawl_data_json():
    for linkdin_url in url_all_page:
        browser.get(linkdin_url)
        sleep(2)
        
        page_source = BeautifulSoup(browser.page_source, "html.parser")
        # Post
        post = page_source.find('span',class_='break-words').get_text()
        # Name post
        name_post = page_source.find('span',class_='feed-shared-actor__name t-14 t-bold hoverable-link-text t-black').get_text().strip()
        # link persion post
        link = page_source.find('a',class_='app-aware-link').get('href')
        
        #SHOW MORE COMMENT
#         try:
#             browser.find_element(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/div/section/div[1]/div/div[5]/div[4]/div[3]/div[2]/button/span').click()
#         except:
#             continue

        all_commment =[]
        all_name_comment = []
        href = []
        content = []
        # get name comment
        all_class_name_comment = page_source.find_all('span',class_='comments-post-meta__name-text hoverable-link-text')
        for name in all_class_name_comment:
            all_name_comment.append(name.get_text().strip())

        # get href comment
        all_class_href_commnet = page_source.find_all('a',class_='ember-view inline-flex overflow-hidden t-16 t-black t-bold tap-target')
        for url_comment in all_class_href_commnet:
            href.append('https://www.linkedin.com'+ str(url_comment.get('href')))
        href = list(filter(None, href))

        # get content comment
        all_class_content_comment = page_source.find_all('div',class_='feed-shared-inline-show-more-text comments-comment-item__inline-show-more-text')
        for class_content in all_class_content_comment:
            content.append(class_content.get_text().strip())

        # map data
        map_all_comment = map(lambda x, y, z: x + str(':') + y + str(':') + z, all_name_comment, href,content)    
        for i in map_all_comment:
            all_commment.append(i)

        # time post 
        time_class = page_source.find('span','feed-shared-actor__sub-description t-12 t-normal t-black--light')
        time_post = time_class.find_all('span')[2].get_text().strip()
        #ID post
        
        RE_URL = re.findall('\d+', linkdin_url)
        for i in RE_URL:
            count_len = len(i)
            if count_len == 19:
                ID_POST = i
           

        my_details = {
            'Social_Network': 'Linkedin',
            'Id': ID_POST,
            'Key_word': key_word,
            'Names': name_post,
            'Link_post': url,
            'post': post,
            'comment': all_commment,
            'device': None,
            'location': None,
            'Job_title': None,
            'time': time_post
        }
        print(my_details)
crawl_data_json()

{'Social_Network': 'Linkedin', 'Id': '6871330718723690496', 'Key_word': 'data engineer', 'Names': 'Indian Institute of Technology, Bombay', 'Link_post': 'https://www.linkedin.com/posts/johnslatterygeaviation_aviation-activity-6871964231655333888-DZJd', 'post': '\nIIT Bombay is pleased to share that its alumnus, Dr. Parag Agrawal, Young Alumni Achiever Awardee (2019), has been appointed the Chief Executive Officer (CEO) of Twitter. Dr. Agrawal obtained his B.Tech. degree in Computer Science and Engineering from IIT Bombay in 2005 and Ph.D. in Computer Science from Stanford University in 2011.Prior to joining Twitter, Dr. Agrawal did research in large-scale data management with collaborators at Microsoft Research, Yahoo! Research and AT&T Labs. He started his journey with Twitter in 2011 as a distinguished software engineer and led efforts on scaling Twitter Ad systems, as well as re-accelerating user growth by improving home timeline relevance. He became the Chief Technical Officer (CTO

KeyboardInterrupt: 

In [327]:
# browser.find_element(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/div/section/div[1]/div/div[5]/div[4]/div[3]/div[2]/button/span').click()
        
browser.find_element(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/div/section/div[1]/div/div[4]/div[4]/div[3]/div[2]/button/span').click()

In [329]:
ID_POST = []
for url in url_all_page:
    RE_URL = re.findall('\d+', url)
    for i in RE_URL:
        count_len = len(i)
        if count_len == 19:
            ID_POST.append(i)
print(ID_POST)
            

['6871330718723690496', '6871669016210108416', '6871789085816561664', '6871671867804045312', '6871282906841563136', '6872175438224011264', '6871790363623874560', '6871472319672446977', '6871459960107126784', '6871410528258981889', '6871966829166813184', '6871431458490220544', '6871209633990766592', '6871853149657858048', '6872241218068217856', '6872193050077212673', '6871622421619740672', '6872161340996231168', '6872058915052515328', '6872028414925164544', '6871624195227430912', '6871946491745976321', '6872011917112832000', '6871613409163649024', '6871703066475950080', '6871474576044429312', '6872172655341109248', '6871685690002960384', '6871523261075853312', '6872228683810177025', '6871254261129453568', '6871824926785970176', '6871453687181570048', '6872202261532164096', '6871320158909026304', '6871897424969957376', '6871971780068827136', '6871851637267320832', '6871853169928941569', '6871964231655333888']


In [348]:
browser.get('https://www.linkedin.com/posts/syedalam011_hiring-recruitment-jobs-activity-6871669016210108416-xE-r/')
sleep(2)

page_source = BeautifulSoup(browser.page_source, "html.parser")
# Post
post = page_source.find('span',class_='break-words').get_text()
# Name post
name_post = page_source.find('span',class_='feed-shared-actor__name t-14 t-bold hoverable-link-text t-black').get_text()
# link persion post
link = page_source.find('a',class_='app-aware-link').get('href')

#SHOW MORE COMMENT
browser.find_element(By.XPATH,'/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/div/section/div[1]/div/div[5]/div[4]/div[3]/div[2]/button/span').click()

all_commment =[]
all_name_comment = []
href = []
content = []
# get name comment
all_class_name_comment = page_source.find_all('span',class_='comments-post-meta__name-text hoverable-link-text')
for name in all_class_name_comment:
    all_name_comment.append(name.get_text().strip())

# get href comment
all_class_href_commnet = page_source.find_all('a',class_='ember-view inline-flex overflow-hidden t-16 t-black t-bold tap-target')
for url_comment in all_class_href_commnet:
    href.append('https://www.linkedin.com'+ str(url_comment.get('href')))
href = list(filter(None, href))

# get content comment
all_class_content_comment = page_source.find_all('div',class_='feed-shared-inline-show-more-text comments-comment-item__inline-show-more-text')
for class_content in all_class_content_comment:
    content.append(class_content.get_text().strip())

# map data
map_all_comment = map(lambda x, y, z: x + str(':') + y + str(':') + z, all_name_comment, href,content)    
for i in map_all_comment:
    all_commment.append(i)

# time post 
time_class = page_source.find('span','feed-shared-actor__sub-description t-12 t-normal t-black--light')
time_post = x.find_all('span')[2].get_text()
#ID post
ID_POST = []

RE_URL = re.findall('\d+', url)
for i in RE_URL:
    count_len = len(i)
    if count_len == 19:
        ID_POST.append(i)


my_details = {
    'Social_Network': 'Linkedin',
    'Id': ID_POST,
    'Key_word': key_word,
    'Names': name_post,
    'Link_post': url,
    'post': post,
    'comment': all_commment,
    'device': None,
    'location': None,
    'Job_title': None,
    'time': time_post
}
print(my_details)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/div/section/div[1]/div/div[5]/div[4]/div[3]/div[2]/button/span"}
  (Session info: chrome=96.0.4664.45)
Stacktrace:
Backtrace:
	Ordinal0 [0x008D6903+2517251]
	Ordinal0 [0x0086F8E1+2095329]
	Ordinal0 [0x00772848+1058888]
	Ordinal0 [0x0079D448+1233992]
	Ordinal0 [0x0079D63B+1234491]
	Ordinal0 [0x007C7812+1406994]
	Ordinal0 [0x007B650A+1336586]
	Ordinal0 [0x007C5BBF+1399743]
	Ordinal0 [0x007B639B+1336219]
	Ordinal0 [0x007927A7+1189799]
	Ordinal0 [0x00793609+1193481]
	GetHandleVerifier [0x00A65904+1577972]
	GetHandleVerifier [0x00B10B97+2279047]
	GetHandleVerifier [0x00966D09+534521]
	GetHandleVerifier [0x00965DB9+530601]
	Ordinal0 [0x00874FF9+2117625]
	Ordinal0 [0x008798A8+2136232]
	Ordinal0 [0x008799E2+2136546]
	Ordinal0 [0x00883541+2176321]
	BaseThreadInitThunk [0x76E36739+25]
	RtlGetFullPathName_UEx [0x77D18AFF+1215]
	RtlGetFullPa

In [370]:
# browser.find_element(By.CLASS_NAME,'search-marvel-srp search-marvel-srp--content').click()
browser.find_element(By.XPATH,'//*[@aria-label="Page 4"]').click()


In [380]:

browser.execute_script("window.scrollTo(0, 1080)") 


In [407]:
for i in range(10):
    sleep(1)
    element = browser.find_element(By.XPATH,f'//*[@id="search-marvel-srp-scroll-container"]/div/div[1]/ul/li[{1+i}]')
    actions = ActionChains(browser)
    actions.move_to_element(element).perform()

    browser.find_element(By.XPATH,f'//*[@id="search-marvel-srp-scroll-container"]/div/div[1]/ul/li[{1+i}]').click()
    sleep(3)
    
    

In [410]:
#     browser.execute_script("arguments[0].scrollIntoView();", element)
element = browser.find_element(By.XPATH,f'//*[@id="search-marvel-srp-scroll-container"]/div/div[1]/ul/li[{1+i}]')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="search-marvel-srp-scroll-container"]/div/div[1]/ul/li[10]"}
  (Session info: chrome=96.0.4664.45)
Stacktrace:
Backtrace:
	Ordinal0 [0x008D6903+2517251]
	Ordinal0 [0x0086F8E1+2095329]
	Ordinal0 [0x00772848+1058888]
	Ordinal0 [0x0079D448+1233992]
	Ordinal0 [0x0079D63B+1234491]
	Ordinal0 [0x007C7812+1406994]
	Ordinal0 [0x007B650A+1336586]
	Ordinal0 [0x007C5BBF+1399743]
	Ordinal0 [0x007B639B+1336219]
	Ordinal0 [0x007927A7+1189799]
	Ordinal0 [0x00793609+1193481]
	GetHandleVerifier [0x00A65904+1577972]
	GetHandleVerifier [0x00B10B97+2279047]
	GetHandleVerifier [0x00966D09+534521]
	GetHandleVerifier [0x00965DB9+530601]
	Ordinal0 [0x00874FF9+2117625]
	Ordinal0 [0x008798A8+2136232]
	Ordinal0 [0x008799E2+2136546]
	Ordinal0 [0x00883541+2176321]
	BaseThreadInitThunk [0x76E36739+25]
	RtlGetFullPathName_UEx [0x77D18AFF+1215]
	RtlGetFullPathName_UEx [0x77D18ACD+1165]


In [445]:
for i in range(10):
    #CLick vao cac post de lay the class

    try:
        element = browser.find_element(By.XPATH,f'//*[@id="search-marvel-srp-scroll-container"]/div/div[1]/ul/li[{1+i}]')
        actions = ActionChains(browser)
        actions.move_to_element(element).perform()

        browser.find_element(By.XPATH,f'//*[@id="search-marvel-srp-scroll-container"]/div/div[1]/ul/li[{1+i}]').click()
        sleep(1)
    except:
        pass
#     #click lay link url post
#     browser.find_element(By.CLASS_NAME,'feed-shared-control-menu__trigger').click()
#     sleep(1)
#     browser.find_element(By.CLASS_NAME,'feed-shared-control-menu__item.option-share-via').click()
#     sleep(1)
#     #tat clipboard de pass qua err
#     browser.find_element(By.CLASS_NAME,'artdeco-button__icon').click()

In [6]:
def get_url_profile():
    all_url_post = []
    #CLick vao cac post de lay the class       
    sleep(1)
    recentList = browser.find_elements(By.CLASS_NAME,"reusable-search__result-container ") 

    list.click()
    sleep(2)

    #click lay link url post
    browser.find_element(By.CLASS_NAME,'feed-shared-control-menu__trigger').click()
    sleep(1)
    browser.find_element(By.CLASS_NAME,'feed-shared-control-menu__item.option-share-via').click()
    sleep(2)
    #tat clipboard de pass qua err
    browser.find_element(By.CLASS_NAME,'artdeco-button__icon').click()
    sleep(1)
    #lay du lieu ra tu clipboard
    win32clipboard.OpenClipboard()
    data = win32clipboard.GetClipboardData()
    win32clipboard.CloseClipboard()
    all_url_post.append(data)


get_url_profile()

AttributeError: type object 'list' has no attribute 'click'

In [33]:
def get_url_all_page():
    number_page = int(input("số trang : "))
    url_all_page = []
    for i in range(number_page):
        print(i+1)
        all_url_post = []
        recentList = browser.find_elements(By.CLASS_NAME, "reusable-search__result-container")
        fBody = browser.find_element(By.ID,"search-marvel-srp-scroll-container")
        try:
            for list in recentList:
                browser.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',
                                       fBody)
                sleep(1)
                list.click()
                sleep(1)

                # click lay link url post
                browser.find_element(By.CLASS_NAME, 'feed-shared-control-menu__trigger').click()
                sleep(1)
                browser.find_element(By.CLASS_NAME, 'feed-shared-control-menu__item.option-share-via').click()
                sleep(2)
                # tat clipboard de pass qua err
                browser.find_element(By.CLASS_NAME, 'artdeco-button__icon').click()

                sleep(1)
                # lay du lieu ra tu clipboard
                win32clipboard.OpenClipboard()
                data = win32clipboard.GetClipboardData()
                win32clipboard.CloseClipboard()
                all_url_post.append(data)

                browser.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',
                                           fBody)
        except:
            pass
        for url in all_url_post:
            url_all_page.append(url)
        sleep(1)
        try:
            browser.find_element(By.XPATH, f'//*[@aria-label="Page {i + 2}"]').click()
        except :
            pass
        sleep(2)
    return url_all_page


In [34]:
def get_url_all_page():
    number_page = int(input("số trang : "))
    url_all_page = []
    for i in range(number_page):
        try:
        try:
            print(i + 1)
            url_one_page = get_url_profile()
            sleep(2)


            # sleep(1)
            # element = browser.find_element(By.CLASS_NAME, 'search-marvel-srp__content-detail-bottom-divider')
            # element.location_once_scrolled_into_view
            sleep(1)
            browser.find_element(By.XPATH, f'//*[@aria-label="Page {i + 2}"]').click()
            sleep(1.5)

            # browser.refresh()
            # sleep(1)

            for url in url_one_page:
                url_all_page.append(url)
            sleep(1)
        except:
            browser.refresh()
            sleep(1)
            browser.find_element(By.XPATH, f'//*[@aria-label="Page {i + 2}"]').click()

    return url_all_page


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


['https://www.linkedin.com/posts/alexei-navolokin-5420563_can-robots-be-taught-artificial-intelligence-activity-6873249943818244096-6OEq',
 'def get_url_profile():\r\n    all_url_post = []\r\n    # CLick vao cac post de lay the class\r\n    sleep(1)\r\n    recentList = browser.find_elements(By.CLASS_NAME, "reusable-search__result-container")\r\n    fBody = browser.find_element(By.ID,"search-marvel-srp-scroll-container")\r\n\r\n    for list in recentList:\r\n        try:\r\n            browser.execute_script(\'arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;\',\r\n                                   fBody)\r\n            sleep(1)\r\n            list.click()\r\n            sleep(1)\r\n\r\n            # click lay link url post\r\n            browser.find_element(By.CLASS_NAME, \'feed-shared-control-menu__trigger\').click()\r\n            sleep(1)\r\n            browser.find_element(By.CLASS_NAME, \'feed-shared-control-menu__item.option-share-via\').click()\r\n  

In [114]:

def get_url_all_page():
    number_page = int(input("số trang : "))
    url_all_page = []
    for i in range(number_page):
        print(i+1)
        all_url_post = []
        recentList = browser.find_elements(By.CLASS_NAME, "reusable-search__result-container")
        fBody = browser.find_element(By.ID,"search-marvel-srp-scroll-container")
  
        for list in recentList:
            try:
                browser.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',
                                       fBody)
                sleep(1)
                list.click()
                sleep(1)

                # click lay link url post
                browser.find_element(By.CLASS_NAME, 'feed-shared-control-menu__trigger').click()
                sleep(1)
                browser.find_element(By.CLASS_NAME, 'feed-shared-control-menu__item.option-share-via').click()
                sleep(2)
                # tat clipboard de pass qua err
                browser.find_element(By.CLASS_NAME, 'artdeco-button__icon').click()

                sleep(1)
                # lay du lieu ra tu clipboard
                win32clipboard.OpenClipboard()
                data = win32clipboard.GetClipboardData()
                win32clipboard.CloseClipboard()
                all_url_post.append(data)

                browser.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',
                                           fBody)
            except:
                continue

        for url in all_url_post:
            url_all_page.append(url)
        sleep(1)
        sleep(1)
        try:
            try:
                browser.find_element(By.XPATH, f'//*[@aria-label="Page {i + 2}"]').click()        

            except:
                fBody = browser.find_element(By.ID,"search-marvel-srp-scroll-container")


                for i in range(3):
                    browser.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',
                                               fBody)
                    sleep(1)
                browser.find_element(By.XPATH, f'//*[@aria-label="Page {i + 2}"]').click()
        except:
            continue
    return url_all_page

get_url_all_page()

số trang : 25
1
2
3
4
5
6
7
8
9
10
11
12


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="search-marvel-srp-scroll-container"]"}
  (Session info: chrome=96.0.4664.93)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F46903+2517251]
	Ordinal0 [0x00EDF8E1+2095329]
	Ordinal0 [0x00DE2848+1058888]
	Ordinal0 [0x00E0D448+1233992]
	Ordinal0 [0x00E0D63B+1234491]
	Ordinal0 [0x00E37812+1406994]
	Ordinal0 [0x00E2650A+1336586]
	Ordinal0 [0x00E35BBF+1399743]
	Ordinal0 [0x00E2639B+1336219]
	Ordinal0 [0x00E027A7+1189799]
	Ordinal0 [0x00E03609+1193481]
	GetHandleVerifier [0x010D5904+1577972]
	GetHandleVerifier [0x01180B97+2279047]
	GetHandleVerifier [0x00FD6D09+534521]
	GetHandleVerifier [0x00FD5DB9+530601]
	Ordinal0 [0x00EE4FF9+2117625]
	Ordinal0 [0x00EE98A8+2136232]
	Ordinal0 [0x00EE99E2+2136546]
	Ordinal0 [0x00EF3541+2176321]
	BaseThreadInitThunk [0x75A76739+25]
	RtlGetFullPathName_UEx [0x77558AFF+1215]
	RtlGetFullPathName_UEx [0x77558ACD+1165]


In [112]:

for x in range(46):
    print(x+1)

    sleep(2)
    fBody = browser.find_element(By.ID,"search-marvel-srp-scroll-container")


    for i in range(3):
        browser.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;',
                                   fBody)
        sleep(1)
    browser.find_element(By.XPATH, f'//*[@aria-label="Page {x + 2}"]').click()

    
    


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


KeyboardInterrupt: 

In [143]:
see_more = browser.find_element(By.CLASS_NAME, 'artdeco-card__action').click()


In [152]:
content = page_source_company.find('div', id='job-details')
post = content.find_all('span')[1].get_text()

print(post)


          WHAT WE CAN OFFER Competitive compensation package and benefits Allowance for academy degrees and language certification, training course, teambuilding.... International working environment & Learning systemsWhat You will get:Exciting opportunities just become available at Hitachi Vantara Vietnam for English speaking freshers who are keen on developing career in IT industry as Data Analyst and working on project for Google, the biggest internet company in the world.What You will do:You will be trained all necessary knowledge and skills to do the following tasks:Data labelling, annotation & transcription of different data types like audio, images and video.Adhere to client SOPs/guidelines on labelling etc in order to meet client outcomes like Quality, Productivity & Utilization.Interpret customer feedback, monitor satisfaction and help drive programs to improve metrics.Timely deliver different project's data at the highest possible quality standard.Help design processes for Q